# Chapter 15: Autoencoders

## Chapter Exercises

### 1. What are the main tasks that autoencoders are used for?

### 2. Suppose you want to train a classifier and have plenty of unlabeled training data, but only a few thousand labeled instances. How could autoencoders help? How would you proceed?

### 3. If an autoencoder perfectly reconstructs the inputs, is it necessarily a good autoencoder? How can you evaluate the performance of the autoencoder?

### 4. What are undercomplete and overcomplete autoencoders? What is the main risk of an excessive undercomplete autoencoder? What about the main risk of an overcomplete autoencoder?

### 5. How would you tie weights in a stacked autoencoder? What is the point of doing so?

### 6. What is a common technique to visualize features learned by the lower layer of a stacked autoencoder? What about the higher layers?

### 7. What is a generative model? Can you name a type of generative autoencoder?

### 8. Let's use a denoising autoencoder to pretrain an image classifier.

#### You can use MNIST (simplest), or another large set of images such as [CIFAR10](https://www.cs.toronto.edu/~kriz/cifar.html) if you want a bigger challenge. If you choose CIFAR10 you need to write code to load batches of images for traiing. If you want to skip this part, [TensorFlow's model zoo contains tools to do just that](https://github.com/tensorflow/models/blob/master/research/slim/download_and_convert_data.py).

#### Split the dataset into a training set and a test set. Train a deep denoising autoencoder on the full training set.

#### Check that the images are fairly well reconstructed, and visualize the low level features. Visualize the images that most activate each neuron in the coding layer.

#### Build a classification deep neural network, reusing the lower layers of the autoencoder. Train it using only 10% of the training set. Can you get it to perform as well as the same classifier trained on the full training set?

### 9. [_Semanting hashing_](http://www.cs.toronto.edu/~rsalakhu/papers/sdarticle.pdf) is a technique used for efficient _information retrieval_: a document (e.g. an iamges) is passed throug a system (typically a neural network) that outputs a low-dimensional binary vector (e.g. 30 bits). Indexing each document using its hash, it is possible to retrieve many documents similar to the particular document almonst instantly. Let's implement semantic hashing using a slightly tweaked stacked autoencoder.

#### Create a stacked autoencoder containng two hidden layers below the coding layer, and train it in the image dataset you used in the previous exercis. The coding layer should contain 30 neurons and use the logistic activation function to output values between 0 and 1. After training, to produce the hash of an image, you can simply run it through the autoencoder, take the output of the coding layer and round the output of each neuron to 0 or 1.

#### One trick proposed by Salakhutdinov and Hinton is to add Gaussian noise (with zero mean) to the inputs of the coding layer during training. In order to preserve a high signal-to-noise ration, the autoencoder will learn to feed large values to the coding layer. In turn, this means the logistic activation function of the coding layer will likely saturate at 0 or 1. As a result, rounding the codings to 0 or 1 won't distort the output as much.

#### Compute the hash of every image, and see if images with identical hashes look alike. Since the dataset is labeled, one way to measure the performance of the autoencoder is to see if images with the same hash are part of the same class. You can use the Gini purity (introduced in Chapter 6) of the sets of images with identical (or very similar hashes).

#### Try fine tuning the hyperparameters using cross-validation.

#### Note that with a labeled dataset, another approach is to train a convolutional neural network (CNN) for classiciation, then use the layer below the output layer to produce the hashes. See [Jimma Gua and Jianmin Li's 2015 paper](https://arxiv.org/pdf/1509.01354.pdf). See if it performs better.

### 10. Train a variational autoencoder on the image dataset used in the previous exercises and make it generate images. Alternatively, you can try to find an unlabeled dataset that you are interested in and see if you can generate new samples.